In [24]:
import pandas as pd

#создание DataFrame с помощью чтения данных из файла
frame = pd.read_csv('checkins.dat', header=0, sep=',')
print frame.info()
frame.latitude=frame.latitude.str.strip()
frame.longitude=frame.longitude.str.strip()
frame = frame[frame.latitude!=""]
frame.latitude = frame.latitude.astype(float)
frame.longitude = frame.longitude.astype(float)
print frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1021966 entries, 0 to 1021965
Data columns (total 6 columns):
id            1021966 non-null int64
user_id       1021966 non-null int64
venue_id      1021966 non-null int64
latitude      1021966 non-null object
longitude     1021966 non-null object
created_at    1021966 non-null object
dtypes: int64(3), object(3)
memory usage: 46.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 396634 entries, 1 to 1021964
Data columns (total 6 columns):
id            396634 non-null int64
user_id       396634 non-null int64
venue_id      396634 non-null int64
latitude      396634 non-null float64
longitude     396634 non-null float64
created_at    396634 non-null object
dtypes: float64(2), int64(3), object(1)
memory usage: 21.2+ MB
None


In [25]:
#отбираем 100000
frame_temp = frame.iloc[:100000]
print frame_temp.shape
print frame_temp.columns

(100000, 6)
Index([u'id', u'user_id', u'venue_id', u'latitude', u'longitude',
       u'created_at'],
      dtype='object')


In [40]:
#кластерный анализ
from sklearn.cluster import MeanShift
test_frame = frame_temp

X = test_frame.loc[:, lambda df: ['latitude', 'longitude']]
y = test_frame.loc[:,"id"]

ms = MeanShift(bandwidth=0.1)
ms.fit(X)
labels = ms.labels_
cluster_centers = ms.cluster_centers_
print labels.shape
print cluster_centers.shape

(100000,)
(3229, 2)


In [41]:
#подсчет объема кластеров и отбор больших (больше 15 элементов)
import collections
counter = collections.Counter(labels)
result = filter(lambda x: x[1]>15, counter.items())
labels_bigclusters = zip(*result)
print len(labels_bigclusters[0])

592


In [42]:
#загрузка городов из задания
carivalFrame = pd.read_csv('carival.csv', header=0, sep=',')
carivalOffice = carivalFrame.loc[:, lambda df: ['latitude', 'longitude']]
print carival.shape

(6, 4)


In [45]:
#ищем 20 кластерных центров наиболее близких к офисам
result_dist = []
countertmp = 0
from numpy import linalg
for tmp in cluster_centers:
    if countertmp in labels_bigclusters[0]:
        result_dist.append ([countertmp, min(linalg.norm(carivalOffice - tmp, axis=1))])  
    countertmp += 1
print result_dist[:20]
result_dist = sorted(result_dist, key=lambda distance: distance[1])
print result_dist[:20]
for tmp in result_dist[:20]:
    print "index %s - center %s - dist %s - power %s" % (tmp[0],cluster_centers[tmp[0]], tmp[1],labels_bigclusters[1][tmp[0]] )

[[0, 16.143719990900713], [1, 6.1939591656013153], [2, 6.2942414627471628], [3, 17.598579819253398], [4, 5.77204797983301], [5, 13.424131381056705], [6, 6.378262487136821], [7, 8.8854260047572584], [8, 18.912201694843972], [9, 14.461227370022126], [10, 3.8345848685390518], [11, 0.31488379033627317], [12, 23.080862869316192], [13, 17.965990500096549], [14, 16.251596163078347], [15, 14.495236935243689], [16, 14.999911112795457], [17, 0.37868750125029754], [18, 1.3516553532143341], [19, 17.848302400837376]]
[[413, 0.0078347581631091553], [375, 0.0093533161859922255], [410, 0.022674066158382036], [58, 0.050058294822787883], [51, 0.070847732427199731], [29, 0.13410903336184438], [166, 0.16740596425033946], [92, 0.18887596060185352], [87, 0.19577945647763628], [42, 0.21181053682436798], [285, 0.22223329073179071], [315, 0.27130075950667704], [119, 0.29497888680045692], [55, 0.30227011869246351], [27, 0.30473050307840693], [11, 0.31488379033627317], [32, 0.33881047025113181], [158, 0.34084565